In [ ]:
# pip install numpy scipy matplotlib soundfile

In [ ]:
import numpy as np
import plotly.graph_objects as go
import soundfile as sf
from scipy.fft import fft, ifft, fftfreq
from plotly.subplots import make_subplots

file_name='DNote'

# Load the audio file
# file_path = 'RecordingTest.mp3'
file_path = file_name+'.mp3'
signal, sample_rate = sf.read(file_path)

# If the audio file is stereo, take only one channel
if signal.ndim > 1:
    signal = signal[:, 0]

# Calculate the time axis
time = np.linspace(0, len(signal) / sample_rate, num=len(signal))

# Perform Fourier transform
N = len(signal)
yf = fft(signal)
xf = fftfreq(N, 1 / sample_rate)

# Filter the frequency data to limit between 0 and 5k Hz
freq_limit = 2000
indices = np.where((xf >= 0) & (xf <= freq_limit))
xf_limited = xf[indices]
yf_limited = 2.0 / N * np.abs(yf[indices])

# Create a subplot grid
fig = make_subplots(rows=1, cols=2, subplot_titles=('Time Domain', 'Frequency Spectrum'))

# Plot the original signal in time domain using 'sampled' mode
fig.add_trace(go.Scatter(x=time, y=signal, mode='lines', name='Time Domain', line_simplify=True), row=1, col=1)

# Plot the frequency spectrum using 'sampled' mode
fig.add_trace(go.Scatter(x=xf_limited, y=yf_limited, mode='lines', name='Frequency Spectrum', line_simplify=True), row=1, col=2)

# Add titles and layout
fig.update_layout(
    title='Audio Signal and Frequency Spectrum',
    showlegend=False,
    legend=dict(x=0.5, y=1.1, xanchor='center', orientation='h')
)

fig.update_xaxes(title_text='Time [s]', row=1, col=1)
fig.update_yaxes(title_text='Amplitude', row=1, col=1)
fig.update_xaxes(title_text='Frequency [Hz]', row=1, col=2, range=[0, freq_limit])
fig.update_yaxes(title_text='Magnitude', row=1, col=2)

# Show the plot
fig.show()



In [ ]:
# Part 2: Reconstruct signal using different number of most important Fourier modes and save audio files

# Focus on frequencies within 0 to 1 kHz
freq_reconstruction_limit = 10000
indices_reconstruction = np.where((xf >= 0) & (xf <= freq_reconstruction_limit))[0]

# Define the number of modes to keep for reconstruction
num_modes_list = [50,500,5000,50000,500000]

# Part 3: Create a plot for each reconstructed signal comparing it to the original one
comparison_figs = []

for num_modes in num_modes_list:
    # Sort the Fourier coefficients by magnitude and select the top num_modes indices
    sorted_indices = np.argsort(np.abs(yf[indices_reconstruction]))[::-1][:num_modes]
    important_indices = indices_reconstruction[sorted_indices]
    
    # Create a mask to keep only the most important frequencies
    mask = np.zeros(N, dtype=bool)
    mask[important_indices] = True
    mask[-important_indices.size:] = True

    # Apply the mask to the Fourier coefficients
    yf_masked = yf * mask

    # Perform the inverse Fourier transform to reconstruct the signal
    signal_reconstructed = np.real(ifft(yf_masked))

    # Save the reconstructed signal as an audio file
    output_file = 'reconstructed_'+file_name+f'{num_modes}_modes.wav'
    sf.write(output_file, signal_reconstructed, sample_rate)

    # Print the status
    print(f'Reconstructed signal with'+file_name+ '{num_modes} most important modes saved as {output_file}')
    
    # Create a comparison plot
    comparison_fig = go.Figure()
    comparison_fig.add_trace(go.Scatter(x=time, y=signal, mode='lines', name='Original Signal', line_simplify=True))
    comparison_fig.add_trace(go.Scatter(x=time, y=signal_reconstructed, mode='lines', name=f'Reconstructed Signal ({num_modes} modes)', line_simplify=True))
    comparison_fig.update_layout(
        title=f'Original Signal vs. Reconstructed Signal ({num_modes} modes)',
        xaxis_title='Time [s]',
        yaxis_title='Amplitude',
        showlegend=True
    )
    
    comparison_figs.append(comparison_fig)

# Show all comparison plots
for comparison_fig in comparison_figs:
    comparison_fig.show()
